In [1]:
# Copyright 2022 Bimghi Choi. All Rights Reserved.

# -*- coding:utf-8 -*-

# 기간별 앙상블들의 수익률 생성
# reinfo, pred_term은 [0, 0.32], [5, 40]에서 random으로 n번 반복
# 예) 2017-12-31 ~ 2021-12-31 의 trained 모델들의 앙상블 수익률 list

import tensorflow as tf
from tensorflow import keras
import profit
import make_reinfo2 as mr2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random
import sys
from datetime import datetime 


ensembles = [

    ['20C', '25HL', '40C'],

]

model_pools = ["5C", "5HL", "5P", "10C", "10HL", "10P", "15C", "15HL", "15P", "20C", "20HL", "20P",
              "25C", "25HL", "25P", "30C", "30HL", "30P", "40C", "40HL", "40P"]

loss_cut = 0.005


In [2]:
last_trains = [
    '2016-12-31',
    '2017-01-15', '2017-01-31', '2017-02-15', '2017-02-28',
    '2017-03-15', '2017-03-31', '2017-04-15', '2017-04-30',
    '2017-05-15', '2017-05-31', '2017-06-15', '2017-06-30',
    '2017-07-15', '2017-07-31', '2017-08-15', '2017-08-31',
    '2017-09-15', '2017-09-30', '2017-10-15', '2017-10-31',
    '2017-11-15', '2017-11-30', '2017-12-15', '2017-12-31',

    '2018-01-15', '2018-01-31', '2018-02-15', '2018-02-28',
    '2018-03-15', '2018-03-31', '2018-04-15', '2018-04-30',
    '2018-05-15', '2018-05-31', '2018-06-15', '2018-06-30',
    '2018-07-15', '2018-07-31', '2018-08-15', '2018-08-31',
    '2018-09-15', '2018-09-30', '2018-10-15', '2018-10-31',
    '2018-11-15', '2018-11-30', '2018-12-15',

    '2018-12-31',
    '2019-01-15', '2019-01-31', '2019-02-15', '2019-02-28',
    '2019-03-15', '2019-03-31', '2019-04-15', '2019-04-30',
    '2019-05-15', '2019-05-31', '2019-06-15', '2019-06-30',
    '2019-07-15', '2019-07-31', '2019-08-15', '2019-08-31',
    '2019-09-15', '2019-09-30', '2019-10-15', '2019-10-31',
    '2019-11-15', '2019-11-30', '2019-12-15',

    '2019-12-31',
    '2020-01-15', '2020-01-31', '2020-02-15', '2020-02-28',
    '2020-03-15', '2020-03-31', '2020-04-15', '2020-04-30',
    '2020-05-15', '2020-05-31', '2020-06-15', '2020-06-30',
    '2020-07-15', '2020-07-31', '2020-08-15', '2020-08-31',
    '2020-09-15', '2020-09-30', '2020-10-15', '2020-10-31',
    '2020-11-15', '2020-11-30', '2020-12-15',

    '2020-12-31',
    '2021-01-15', '2021-01-31', '2021-02-15', '2021-02-28',
    '2021-03-15', '2021-03-31', '2021-04-15', '2021-04-30',
    '2021-05-15', '2021-05-31', '2021-06-15', '2021-06-30',
    '2021-07-15', '2021-07-31', '2021-08-15', '2021-08-31',
    '2021-09-15', '2021-09-30', '2021-10-15', '2021-10-31',
    '2021-11-15', '2021-11-30', '2021-12-15',

    '2021-12-31',
    '2022-01-15', '2022-01-31', '2022-02-15', '2022-02-28',
    '2022-03-15', '2022-03-31', '2022-04-15', '2022-04-30',
    '2022-05-15', '2022-05-31', '2022-06-15', '2022-06-30',
    '2022-07-15', '2022-07-31', '2022-08-15', '2022-08-31',
    '2022-09-15', '2022-09-30', '2022-10-15', '2022-10-31',
    '2022-11-15', '2022-11-30', '2022-12-15',

    '2022-12-31',
    '2023-01-15', '2023-01-31', '2023-02-15', '2023-02-28',
    '2023-03-15', '2023-03-31', '2023-04-15', '2023-04-30',
    '2023-05-15', '2023-05-31', '2023-06-15', '2023-06-30',
    '2023-07-15', '2023-07-31', '2023-08-15', '2023-08-31',
    '2023-09-15', '2023-09-30', '2023-10-15', '2023-10-31',
    '2023-11-15', '2023-11-30', '2023-12-15',

    '2023-12-31',
    '2024-01-15', '2024-01-31', '2024-02-15', '2024-02-29',
    '2024-03-15', '2024-03-31', '2024-04-15', '2024-04-30',
    '2024-05-15', '2024-05-31', '2024-06-15', '2024-06-30',
    '2024-07-15', '2024-07-31', '2024-08-15', '2024-08-31',
    '2024-09-15', '2024-09-30', '2024-10-15', '2024-10-31',
    '2024-11-15', '2024-11-30', '2024-12-15',
]

start_times = [
    '2017/01/01/09:00', '2017/01/16/09:00', '2017/02/01/09:00', '2017/02/16/09:00',
    '2017/03/01/09:00', '2017/03/16/09:00', '2017/04/01/09:00', '2017/04/16/09:00',
    '2017/05/01/09:00', '2017/05/16/09:00', '2017/06/01/09:00', '2017/06/16/09:00',
    '2017/07/01/09:00', '2017/07/16/09:00', '2017/08/01/09:00', '2017/08/16/09:00',
    '2017/09/01/09:00', '2017/09/16/09:00', '2017/10/01/09:00', '2017/10/16/09:00',
    '2017/11/01/09:00', '2017/11/16/09:00', '2017/12/01/09:00', '2017/12/16/09:00',

    '2018/01/01/09:00', '2018/01/16/09:00', '2018/02/01/09:00', '2018/02/16/09:00',
    '2018/03/01/09:00', '2018/03/16/09:00', '2018/04/01/09:00', '2018/04/16/09:00',
    '2018/05/01/09:00', '2018/05/16/09:00', '2018/06/01/09:00', '2018/06/16/09:00',
    '2018/07/01/09:00', '2018/07/16/09:00', '2018/08/01/09:00', '2018/08/16/09:00',
    '2018/09/01/09:00', '2018/09/16/09:00', '2018/10/01/09:00', '2018/10/16/09:00',
    '2018/11/01/09:00', '2018/11/16/09:00', '2018/12/01/09:00', '2018/12/16/09:00',

    '2019/01/01/09:00', '2019/01/16/09:00', '2019/02/01/09:00', '2019/02/16/09:00',
    '2019/03/01/09:00', '2019/03/16/09:00', '2019/04/01/09:00', '2019/04/16/09:00',
    '2019/05/01/09:00', '2019/05/16/09:00', '2019/06/01/09:00', '2019/06/16/09:00',
    '2019/07/01/09:00', '2019/07/16/09:00', '2019/08/01/09:00', '2019/08/16/09:00',
    '2019/09/01/09:00', '2019/09/16/09:00', '2019/10/01/09:00', '2019/10/16/09:00',
    '2019/11/01/09:00', '2019/11/16/09:00', '2019/12/01/09:00', '2019/12/16/09:00',

    '2020/01/01/09:00', '2020/01/16/09:00', '2020/02/01/09:00', '2020/02/16/09:00',
    '2020/03/01/09:00', '2020/03/16/09:00', '2020/04/01/09:00', '2020/04/16/09:00',
    '2020/05/01/09:00', '2020/05/16/09:00', '2020/06/01/09:00', '2020/06/16/09:00',
    '2020/07/01/09:00', '2020/07/16/09:00', '2020/08/01/09:00', '2020/08/16/09:00',
    '2020/09/01/09:00', '2020/09/16/09:00', '2020/10/01/09:00', '2020/10/16/09:00',
    '2020/11/01/09:00', '2020/11/16/09:00', '2020/12/01/09:00', '2020/12/16/09:00',

    '2021/01/01/09:00', '2021/01/16/09:00', '2021/02/01/09:00', '2021/02/16/09:00',
    '2021/03/01/09:00', '2021/03/16/09:00', '2021/04/01/09:00', '2021/04/16/09:00',
    '2021/05/01/09:00', '2021/05/16/09:00', '2021/06/01/09:00', '2021/06/16/09:00',
    '2021/07/01/09:00', '2021/07/16/09:00', '2021/08/01/09:00', '2021/08/16/09:00',
    '2021/09/01/09:00', '2021/09/16/09:00', '2021/10/01/09:00', '2021/10/16/09:00',
    '2021/11/01/09:00', '2021/11/16/09:00', '2021/12/01/09:00', '2021/12/16/09:00',

    '2022/01/01/09:00', '2022/01/16/09:00', '2022/02/01/09:00', '2022/02/16/09:00',
    '2022/03/01/09:00', '2022/03/16/09:00', '2022/04/01/09:00', '2022/04/16/09:00',
    '2022/05/01/09:00', '2022/05/16/09:00', '2022/06/01/09:00', '2022/06/16/09:00',
    '2022/07/01/09:00', '2022/07/16/09:00', '2022/08/01/09:00', '2022/08/16/09:00',
    '2022/09/01/09:00', '2022/09/16/09:00', '2022/10/01/09:00', '2022/10/16/09:00',
    '2022/11/01/09:00', '2022/11/16/09:00', '2022/12/01/09:00', '2022/12/16/09:00',

    '2023/01/01/09:00', '2023/01/16/09:00', '2023/02/01/09:00', '2023/02/16/09:00',
    '2023/03/01/09:00', '2023/03/16/09:00', '2023/04/01/09:00', '2023/04/16/09:00',
    '2023/05/01/09:00', '2023/05/16/09:00', '2023/06/01/09:00', '2023/06/16/09:00',
    '2023/07/01/09:00', '2023/07/16/09:00', '2023/08/01/09:00', '2023/08/16/09:00',
    '2023/09/01/09:00', '2023/09/16/09:00', '2023/10/01/09:00', '2023/10/16/09:00',
    '2023/11/01/09:00', '2023/11/16/09:00', '2023/12/01/09:00', '2023/12/16/09:00',

    '2024/01/01/09:00', '2024/01/16/09:00', '2024/02/01/09:00', '2024/02/16/09:00',
    '2024/03/01/09:00', '2024/03/16/09:00', '2024/04/01/09:00', '2024/04/16/09:00',
    '2024/05/01/09:00', '2024/05/16/09:00', '2024/06/01/09:00', '2024/06/16/09:00',
    '2024/07/01/09:00', '2024/07/16/09:00', '2024/08/01/09:00', '2024/08/16/09:00',
    '2024/09/01/09:00', '2024/09/16/09:00', '2024/10/01/09:00', '2024/10/16/09:00',
    '2024/11/01/09:00', '2024/11/16/09:00', '2024/12/01/09:00', '2024/12/16/09:00',
]

end_times = [
    '2017/01/15/15:00', '2017/01/31/15:00', '2017/02/15/15:00', '2017/02/28/15:00',
    '2017/03/15/15:00', '2017/03/31/15:00', '2017/04/15/15:00', '2017/04/30/15:00',
    '2017/05/15/15:00', '2017/05/31/15:00', '2017/06/15/15:00', '2017/06/30/15:00',
    '2017/07/15/15:00', '2017/07/31/15:00', '2017/08/15/15:00', '2017/08/31/15:00',
    '2017/09/15/15:00', '2017/09/30/15:00', '2017/10/15/15:00', '2017/10/31/15:00',
    '2017/11/15/15:00', '2017/11/30/15:00', '2017/12/15/15:00', '2017/12/31/15:00',

    '2018/01/15/15:00', '2018/01/31/15:00', '2018/02/15/15:00', '2018/02/28/15:00',
    '2018/03/15/15:00', '2018/03/31/15:00', '2018/04/15/15:00', '2018/04/30/15:00',
    '2018/05/15/15:00', '2018/05/31/15:00', '2018/06/15/15:00', '2018/06/30/15:00',
    '2018/07/15/15:00', '2018/07/31/15:00', '2018/08/15/15:00', '2018/08/31/15:00',
    '2018/09/15/15:00', '2018/09/30/15:00', '2018/10/15/15:00', '2018/10/31/15:00',
    '2018/11/15/15:00', '2018/11/30/15:00', '2018/12/15/15:00', '2018/12/31/15:00',

    '2019/01/15/15:00', '2019/01/31/15:00', '2019/02/15/15:00', '2019/02/28/15:00',
    '2019/03/15/15:00', '2019/03/31/15:00', '2019/04/15/15:00', '2019/04/30/15:00',
    '2019/05/15/15:00', '2019/05/31/15:00', '2019/06/15/15:00', '2019/06/30/15:00',
    '2019/07/15/15:00', '2019/07/31/15:00', '2019/08/15/15:00', '2019/08/31/15:00',
    '2019/09/15/15:00', '2019/09/30/15:00', '2019/10/15/15:00', '2019/10/31/15:00',
    '2019/11/15/15:00', '2019/11/30/15:00', '2019/12/15/15:00', '2019/12/31/15:00',

    '2020/01/15/15:00', '2020/01/31/15:00', '2020/02/15/15:00', '2020/02/28/15:00',
    '2020/03/15/15:00', '2020/03/31/15:00', '2020/04/15/15:00', '2020/04/30/15:00',
    '2020/05/15/15:00', '2020/05/31/15:00', '2020/06/15/15:00', '2020/06/30/15:00',
    '2020/07/15/15:00', '2020/07/31/15:00', '2020/08/15/15:00', '2020/08/31/15:00',
    '2020/09/15/15:00', '2020/09/30/15:00', '2020/10/15/15:00', '2020/10/31/15:00',
    '2020/11/15/15:00', '2020/11/30/15:00', '2020/12/15/15:00', '2020/12/31/15:00',

    '2021/01/15/15:00', '2021/01/31/15:00', '2021/02/15/15:00', '2021/02/28/15:00',
    '2021/03/15/15:00', '2021/03/31/15:00', '2021/04/15/15:00', '2021/04/30/15:00',
    '2021/05/15/15:00', '2021/05/31/15:00', '2021/06/15/15:00', '2021/06/30/15:00',
    '2021/07/15/15:00', '2021/07/31/15:00', '2021/08/15/15:00', '2021/08/31/15:00',
    '2021/09/15/15:00', '2021/09/30/15:00', '2021/10/15/15:00', '2021/10/31/15:00',
    '2021/11/15/15:00', '2021/11/30/15:00', '2021/12/15/15:00', '2021/12/31/15:00',

    '2022/01/15/15:00', '2022/01/31/15:00', '2022/02/15/15:00', '2022/02/28/15:00',
    '2022/03/15/15:00', '2022/03/31/15:00', '2022/04/15/15:00', '2022/04/30/15:00',
    '2022/05/15/15:00', '2022/05/31/15:00', '2022/06/15/15:00', '2022/06/30/15:00',
    '2022/07/15/15:00', '2022/07/31/15:00', '2022/08/15/15:00', '2022/08/31/15:00',
    '2022/09/15/15:00', '2022/09/30/15:00', '2022/10/15/15:00', '2022/10/31/15:00',
    '2022/11/15/15:00', '2022/11/30:15:00', '2022/12/15/15:00', '2022/12/31/15:00',

    '2023/01/15/15:00', '2023/01/31/15:00', '2023/02/15/15:00', '2023/02/28/15:00',
    '2023/03/15/15:00', '2023/03/31/15:00', '2023/04/15/15:00', '2023/04/30/15:00',
    '2023/05/15/15:00', '2023/05/31/15:00', '2023/06/15/15:00', '2023/06/30/15:00',
    '2023/07/15/15:00', '2023/07/31/15:00', '2023/08/15/15:00', '2023/08/31/15:00',
    '2023/09/15/15:00', '2023/09/30/15:00', '2023/10/15/15:00', '2023/10/31/15:00',
    '2023/11/15/15:00', '2023/11/30/15:00', '2023/12/15/15:00', '2023/12/31/15:00',

    '2024/01/15/15:00', '2024/01/31/15:00', '2024/02/15/15:00', '2024/02/29/15:00',
    '2024/03/15/15:00', '2024/03/31/15:00', '2024/04/15/15:00', '2024/04/30/15:00',
    '2024/05/15/15:00', '2024/05/31/15:00', '2024/06/15/15:00', '2024/06/30/15:00',
    '2024/07/15/15:00', '2024/07/31/15:00', '2024/08/15/15:00', '2024/08/31/15:00',
    '2024/09/15/15:00', '2024/09/30/15:00', '2024/10/15/15:00', '2024/10/24/15:00',
    '2024/11/15/15:00', '2024/11/30/15:00', '2024/12/15/15:00', '2024/12/31/15:00',
]

In [3]:
def set_path(folder_name, result_folder):
    folder = folder_name + "/" + folder_name + "_" + term + "_losscut" + str(loss_cut)
    if not os.path.isdir(folder):
        os.makedirs(folder)

    if not os.path.isdir(result_folder):
        os.makedirs(result_folder)
    result_path = result_folder + "/" + result_folder + "_" + term + "_losscut" + str(loss_cut) + ".csv"

    return folder, result_path

def set_term(start_times, end_times, last_trains, start_time, end_time, add_txt):
    start_index = start_times.index(start_time)
    end_index = end_times.index(end_time)

    last_trains = last_trains[start_index:end_index+1]
    start_times = start_times[start_index:end_index+1]
    end_times = end_times[start_index:end_index+1]

    term = datetime.strptime(start_time, "%Y/%m/%d/%H:%M").strftime("%Y-%m-%d") + "~" + \
           datetime.strptime(end_time, "%Y/%m/%d/%H:%M").strftime("%Y-%m-%d") + add_txt

    return last_trains, start_times, end_times, term

def preprocessing(df0_path, df_pred_path, start_time, end_time):
    # 필요 구간의 전처리 데이터 존재여부에 따라 처리

    if not os.path.isfile(df_pred_path):
        print("==============================================")
    else:
        norm_df0 = pd.read_csv(df_pred_path, encoding='euc-kr')

        df0 = pd.read_csv(df0_path, encoding='euc-kr')
        _end_time = df0.loc[df0['date'] <= end_time].max()['date']
        if start_time > df0['date'].values[-1]:
            start_time = df0['date'].values[-1]
        _start_time = df0.loc[df0['date'] >= start_time].min()['date']
        start_date = norm_df0.loc[norm_df0['date'].index.min(), 'date']
        last_date = norm_df0.loc[norm_df0['date'].index.max(), 'date']


        if last_date >= _end_time and start_date <= _start_time:
            print('nothing done! in this preprocessing')
            return

    df0 = pd.read_csv(df0_path, encoding='euc-kr')

    df0["시가대비종가변화율"] = (df0["종가"] - df0["시가"])/df0["시가"]*100
    df0["시가대비고가변화율"] = (df0["고가"] - df0["시가"])/df0["시가"]*100
    df0["시가대비저가변화율"] = (df0["저가"] - df0["시가"])/df0["시가"]*100
    df0["종가대비고가변화율"] = (df0["고가"] - df0["종가"])/df0["종가"]*100
    df0["종가대비저가변화율"] = (df0["저가"] - df0["종가"])/df0["종가"]*100

    df0["1일전"] = np.concatenate([[0], df0["종가"].values[:-1]])
    df0["2일전"] = np.concatenate([[0, 0], df0["종가"].values[:-2]])
    df0["3일전"] = np.concatenate([[0, 0, 0], df0["종가"].values[:-3]])
    df0["4일전"] = np.concatenate([[0, 0, 0, 0], df0["종가"].values[:-4]])
    df0["5일전"] = np.concatenate([[0, 0, 0, 0, 0], df0["종가"].values[:-5]])

    df0["6일전"] = np.concatenate([[0, 0, 0, 0, 0, 0], df0["종가"].values[:-6]])
    df0["7일전"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0], df0["종가"].values[:-7]])
    df0["8일전"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0, 0], df0["종가"].values[:-8]])
    df0["9일전"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0, 0, 0], df0["종가"].values[:-9]])
    df0["10일전"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], df0["종가"].values[:-10]])

    df0["1일수익률"] = df0["종가"].rolling(window=2).apply(lambda x: x[1] - x[0])
    df0["3일수익률"] = df0["종가"].rolling(window=4).apply(lambda x: x[3] - x[0])
    df0["5일수익률"] = df0["종가"].rolling(window=6).apply(lambda x: x[5] - x[0])
    df0["10일수익률"] = df0["종가"].rolling(window=11).apply(lambda x: x[10] - x[0])
    df0["20일수익률"] = df0["종가"].rolling(window=21).apply(lambda x: x[20] - x[0])
    df0["40일수익률"] = df0["종가"].rolling(window=41).apply(lambda x: x[40] - x[0])
    df0["60일수익률"] = df0["종가"].rolling(window=61).apply(lambda x: x[60] - x[0])
    df0["90일수익률"] = df0["종가"].rolling(window=91).apply(lambda x: x[90] - x[0])
    df0["120일수익률"] = df0["종가"].rolling(window=121).apply(lambda x: x[120] - x[0])
    df0["180일수익률"] = df0["종가"].rolling(window=181).apply(lambda x: x[180] - x[0])
    df0["240일수익률"] = df0["종가"].rolling(window=241).apply(lambda x: x[240] - x[0])

    df0["5일최고"] = df0["고가"].rolling(window=5).max()
    df0["20일최고"] = df0["고가"].rolling(window=20).max()
    df0["60일최고"] = df0["고가"].rolling(window=60).max()
    df0["120일최고"] = df0["고가"].rolling(window=120).max()
    df0["240일최고"] = df0["고가"].rolling(window=240).max()

    df0["5일최저"] = df0["저가"].rolling(window=5).min()
    df0["20일최저"] = df0["저가"].rolling(window=20).min()
    df0["60일최저"] = df0["저가"].rolling(window=60).min()
    df0["120일최저"] = df0["저가"].rolling(window=120).min()
    df0["240일최저"] = df0["저가"].rolling(window=240).min()

    df0["1일전거래량"] = np.concatenate([[0], df0["거래량"].values[:-1]])
    df0["2일전거래량"] = np.concatenate([[0, 0], df0["거래량"].values[:-2]])
    df0["3일전거래량"] = np.concatenate([[0, 0, 0], df0["거래량"].values[:-3]])
    df0["4일전거래량"] = np.concatenate([[0, 0, 0, 0], df0["거래량"].values[:-4]])
    df0["5일전거래량"] = np.concatenate([[0, 0, 0, 0, 0], df0["거래량"].values[:-5]])

    df0["6일전거래량"] = np.concatenate([[0, 0, 0, 0, 0, 0], df0["거래량"].values[:-6]])
    df0["7일전거래량"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0], df0["거래량"].values[:-7]])
    df0["8일전거래량"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0, 0], df0["거래량"].values[:-8]])
    df0["9일전거래량"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0, 0, 0], df0["거래량"].values[:-9]])
    df0["10일전거래량"] = np.concatenate([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], df0["거래량"].values[:-10]])

    df0["1일거래변화량"] = df0["거래량"].rolling(window=2).apply(lambda x: x[1] - x[0])
    df0["3일거래변화량"] = df0["거래량"].rolling(window=4).apply(lambda x: x[3] - x[0])
    df0["5일거래변화량"] = df0["거래량"].rolling(window=6).apply(lambda x: x[5] - x[0])
    df0["10일거래변화량"] = df0["거래량"].rolling(window=11).apply(lambda x: x[10] - x[0])
    df0["20일거래변화량"] = df0["거래량"].rolling(window=21).apply(lambda x: x[20] - x[0])
    df0["40일거래변화량"] = df0["거래량"].rolling(window=41).apply(lambda x: x[40] - x[0])
    df0["60일거래변화량"] = df0["거래량"].rolling(window=61).apply(lambda x: x[60] - x[0])
    df0["90일거래변화량"] = df0["거래량"].rolling(window=91).apply(lambda x: x[90] - x[0])
    df0["120일거래변화량"] = df0["거래량"].rolling(window=121).apply(lambda x: x[120] - x[0])
    df0["180일거래변화량"] = df0["거래량"].rolling(window=181).apply(lambda x: x[180] - x[0])
    df0["240일거래변화량"] = df0["거래량"].rolling(window=241).apply(lambda x: x[240] - x[0])

    df0["5일평균거래량"] = df0["거래량"].rolling(window=5).mean()
    df0["20일평균거래량"] = df0["거래량"].rolling(window=20).mean()
    df0["60일평균거래량"] = df0["거래량"].rolling(window=60).mean()
    df0["120일평균거래량"] = df0["거래량"].rolling(window=120).mean()
    df0["240일평균거래량"] = df0["거래량"].rolling(window=240).mean()

    df0["5일최고거래량"] = df0["거래량"].rolling(window=5).max()
    df0["20일최고거래량"] = df0["거래량"].rolling(window=20).max()
    df0["60일최고거래량"] = df0["거래량"].rolling(window=60).max()
    df0["120일최고거래량"] = df0["거래량"].rolling(window=120).max()
    df0["240일최고거래량"] = df0["거래량"].rolling(window=240).max()

    df0["5일최저거래량"] = df0["거래량"].rolling(window=5).min()
    df0["20일최저거래량"] = df0["거래량"].rolling(window=20).min()
    df0["60일최저거래량"] = df0["거래량"].rolling(window=60).min()
    df0["120일최저거래량"] = df0["거래량"].rolling(window=120).min()
    df0["240일최저거래량"] = df0["거래량"].rolling(window=240).min()


    start_index = df0.loc[df0['date'] >= start_time].index.min()
    end_index = df0.loc[df0['date'] <= end_time].index.max()

    df = df0[start_index - (20-1) : end_index + 1].reset_index(drop=True)
    norm_df = df[20-1 : ].reset_index(drop=True).copy()

    for i in range(end_index - start_index + 1):
        for j in range(1, 83+1):
            m = df.iloc[i:i+20, j].mean()
            s = df.iloc[i:i+20, j].std()
            if s == 0:
                norm_df.iloc[i, j] = 0
            else:
                norm_df.iloc[i, j] = (df.iloc[i+20-1, j] - m) / s
    norm_df.to_csv(df_pred_path, index=False, encoding='euc-kr')
    
def predict(df0_path, df_pred_path, result_path, start_time, end_time, last_train, reinfo_th, reinfo_width, pred_term, target_type, selected_checkpoint_path):
    # create prediction values
    df_pred = pd.read_csv(df_pred_path, encoding='euc-kr')

    if start_time > df_pred['date'].values[-1]:
        start_time = df_pred['date'].values[-1]

    start_index = df_pred.loc[df_pred['date'] >= start_time].index.min()
    end_index = df_pred.loc[df_pred['date'] <= end_time].index.max()

    dates = df_pred.pop('date').values[start_index:end_index + 1].reshape(-1)

    #개별 모델 예측 전 앙상블 pred_term, target_type save하고 개별 모델 예측후 복원
    save_term = pred_term
    save_type = target_type

    #모델들의 예측값 생성
    model = create_model(last_train, pred_term, target_type)
    r = []
    for i in range(3):

        model.load_weights(selected_checkpoint_path[i])

        model_pred_term, model_target_type = parsing(selected_checkpoint_path[i])

        pred = model_predict(model, df0_path, df_pred_path, result_path, start_time, end_time, last_train, reinfo_th, reinfo_width, model_pred_term, model_target_type)[:, 1]

        r.append(pred)

    r = np.array(r)

    pred_term = save_term
    target_type = save_type

    #앙상블의 예측값 생성
    pred = []
    for i in range(len(r[0])):
        cnt = [0, 0, 0]
        for j in range(3):
            cnt[r[j][i]] += 1
        pred.append(np.argmax(cnt))

    # 시가, 고가, 저가, 종가 검색
    df = pd.read_csv(df0_path, encoding='euc-kr')
    start_index = df.loc[df['date'] >=start_time].index.min()
    end_index = df.loc[df['date'] <= end_time].index.max()
    high = df['고가'].values[start_index:end_index + 1]
    low = df['저가'].values[start_index:end_index + 1]
    close = df['종가'].values[start_index:end_index + 1]
    open = df['시가'].values[start_index:end_index + 1]
    #  0: 고점, 1: 저점

    pred_results = []
    for i in range(len(pred)):
        pred_results.append([dates[i], pred[i], open[i], high[i], low[i], close[i]])
    pred_results = np.array(pred_results)

    if reinfo_th > 0:
        mr.th = reinfo_th
        mr.pred_term = pred_term
        mr.target_type = target_type
        pred = mr.reinfo(pred, pred_results, start_time, reinfo_width)
        pred_results[:, 1] = np.array(pred)


    # 결과 파일에 저장
    # 0: 정상, 1: 고점 2:저점
    pd.DataFrame(pred_results, columns=['date', 'result', 'open', 'high', 'low', 'close']).to_csv(result_path, index=False, encoding='euc-kr')

    # 수익률 계산하여 return
    profit.loss_cut = loss_cut
    profit.pred_term = pred_term
    profit.result_path = result_path
    return profit.calc_profit()

def parsing(path):

    start = path.find('/60M_') + 5
    end = path.find('_best')
    model_type = path[start:end]

    c = model_type.find('C')
    h = model_type.find('HL')
    p = model_type.find('P')
    if c != -1:
        pred_term = int(model_type[:c])
        target_type = 'C'
    elif h != -1:
        pred_term = int(model_type[:h])
        target_type = 'HL'
    elif p != -1:
        pred_term = int(model_type[:p])
        target_type = 'P'
    else:
        print("argument error " + model_type)
        exit(0)

    return pred_term, target_type

def create_model(last_train, pred_term, target_type):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(83)),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(int(200 / 2), activation='relu'),
        tf.keras.layers.Dense(int(200 / 4), activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    #cp_callback = tf.keras.callbacks.ModelCheckpoint(
    #    checkpoint_path, verbose=1, save_weights_only=True,
        # 다섯 번째 에포크마다 가중치를 저장합니다
    #    save_freq=5)

    model.compile(optimizer='adam',
                  loss=keras.losses.sparse_categorical_crossentropy,
                  #callbacks=[cp-callback]
                  metrics=['accuracy'])

    checkpoint_path = last_train+"/60M_input83_test"
    checkpoint_path_best = last_train+"/60M_"+str(pred_term) + target_type + "_best"

    model.save_weights(checkpoint_path)

    return model

def model_predict(model, df0_path, df_pred_path, result_path, start_time, end_time, last_train, reinfo_th, reinfo_width, pred_term, target_type):

    df_pred = pd.read_csv(df_pred_path, encoding='euc-kr')

    if start_time > df_pred['date'].values[-1]:
        start_time = df_pred['date'].values[-1]

    start_index = df_pred.loc[df_pred['date'] >= start_time].index.min()
    end_index = df_pred.loc[df_pred['date'] <= end_time].index.max()

    dates = df_pred.pop('date').values[start_index:end_index + 1].reshape(-1)

    pred_input = df_pred.values[start_index:end_index+1, :83].reshape(-1, 83)

    #model.load_weights(checkpoint_path_best)
    pred = model.predict(pred_input)
    pred = np.argmax(pred, axis=1).reshape(-1)

    # 종가 검색
    df = pd.read_csv(df0_path, encoding='euc-kr')
    start_index = df.loc[df['date'] >= start_time].index.min()
    end_index = df.loc[df['date'] <= end_time].index.max()
    high = df['고가'].values[start_index:end_index + 1]
    low = df['저가'].values[start_index:end_index + 1]
    close = df['종가'].values[start_index:end_index + 1]
    open = df['시가'].values[start_index:end_index + 1]

    #  0: 고점, 1: 저점
    pred_results = [dates, pred, open, high, low, close]
    pred_results = np.array(pred_results).transpose()

    if reinfo_th > 0:
        mr.th = reinfo_th
        mr.pred_term = pred_term
        mr.target_type = target_type
        pred = mr.reinfo(pred, pred_results, start_time, reinfo_width)
        pred_results[:, 1] = pred

    # 앙상불 예측(ensemble.py)의 경우 개별 모델들의 result_path에 last_train 폴더를 붙이기 위해서.
    # 이는 자동 거래시 앙상블 예측의 결과가 끝나기 전에 모델의 결과를 앙상블의 결과로 읽는 오류를 막기 위해서이다.
    if result_path.find(last_train) < 0:
        result_path = last_train + "/" + result_path
    else:
        result_path = result_path

    pd.DataFrame(np.array(pred_results), columns=['date', 'result', 'open', 'high', 'low', 'close']).to_csv(result_path, index=False, encoding='euc-kr')
    return pred_results[len(pred_results)-len(dates):]

def calc_MDD(profit_rates):

    MDD = []
    for i in range(len(profit_rates)):
        MDD.append((profit_rates[:i+1].max() - profit_rates[i]) / profit_rates[:i+1].max())
    return np.array(MDD).max()

In [4]:
def main(selected_model_types, folder, pred_term, target_type, reinfo_th, reinfo_width):
    ensemble_models = ""
    for i in range(len(selected_model_types)):
        ensemble_models = ensemble_models + selected_model_types[i] + "_"

    path = folder + "/9시거래없음_ensemble_" + str(pred_term) + target_type + "_reinfo_" + \
           str(reinfo_th) + "_width_" + str(reinfo_width) + "_" + ensemble_models + ".xlsx"

    # 모델별 각 구간별 수익률 계산, 저장
    profit_sum = []
    #profit_product = 1
    profits = np.empty(0)
    dates = np.empty(0)
    closes = np.empty(0)
    for i in range(len(last_trains)):
        last_train = last_trains[i]
        start_time = start_times[i]
        end_time = end_times[i]

        df0_path = 'kospi200f_11_60M.csv' # 원본 파일
        df_raw_path = last_train+'/kospi200f_60M_raw.csv' # 원본 파일에 target값 append
        norm_df_path = last_train+'/kospi200f_60M_norm.csv' # 학습용 normalization file
        df_pred_path = last_train+'/kospi200f_60M_pred.csv' # 예측용 normalization file
        result_path = last_train+'/pred_83_results.csv' # 예측 결과 손익 파일


        preprocessing(df0_path, df_pred_path, start_time, end_time)

        selected_checkpoint_path = ['' for i in range(3)]
        for j in range(3):
            selected_checkpoint_path[j] = last_train + "/60M_" + selected_model_types[j] + "_best"

        p = predict(df0_path, df_pred_path, result_path, start_time, end_time, last_train, reinfo_th, reinfo_width, pred_term, target_type, selected_checkpoint_path)
        print(last_trains[i] + " 수익률: " + str(p))
        profit_sum.append(p)
        #profit_product *= p

        df = pd.read_csv(result_path, encoding='euc-kr')
        profit = df['profit'].values - df['fee'].values
        profits = np.concatenate([profits, profit], axis=0)
        dates = np.concatenate([dates, df['date'].values], axis=0)
        closes = np.concatenate([closes, df['close'].values], axis=0)

    profit_rates = profits.cumsum() / margin + 1
    close_rates = np.array(closes) / closes[0]
    profit_product = np.array(profit_sum).prod()

    print("최종 수익률", profit_rates[-1])
    print("복리누적: " + str(profit_product))
    MDD = calc_MDD(profit_rates)
    print("MDD: ", MDD)

    print(path)

    return dates, closes, profits, close_rates, profit_rates, MDD, profit_product, ensemble_models, path


In [5]:
##################################################################################################################
# 21개 모델 중 3개 선택, reinof_th, pred_term, width 랜덤으로 선택, 100번 실행하여 수익률, MDD, 기하 수익률 저장
##################################################################################################################

loss_cut = 0.005
cnt = 100

start_time = '2023/10/16/09:00'
end_time = '2024/10/24/15:00'
term1 = start_time[:10].replace('/', '-') + "~" + end_time[:10].replace('/', '-')

mr = mr2
profit.slippage = 0.0
margin = 12000000
profit.margin = 120000000
add_txt = "_천2백투자_slippage" + str(profit.slippage)

last_trains, start_times, end_times, term = set_term(start_times, end_times, last_trains, start_time, end_time, add_txt)
folder, result_path = set_path("앙상블실험_random", "eval_reflection_random")

if not os.path.isfile(result_path):
    results = []
else:
    results = pd.read_csv(result_path).values.tolist()
best = [0, 0, 0, '', 0, 0, 0]
while len(results) < cnt:
    # 0.24, 10, 60
    reinfo_th = random.randint(0, 32) / 100
    pred_term = random.randint(5, 40)
    reinfo_width = random.randint(10, 77)
    target_type = 'C'

    for enm in ensembles:

        selected_model_types = sorted(random.sample(model_pools, 3))

        ensemble_models = '["'
        for i in range(2):
            ensemble_models = ensemble_models + selected_model_types[i] + '", "'
        ensemble_models = ensemble_models + selected_model_types[2] + '", ' + str(reinfo_th) + ', ' + str(
            pred_term) + ', ' + str(reinfo_width) + ']'
        if not results == [] and [ensemble_models, str(reinfo_th), str(pred_term), str(reinfo_width)] in np.array(results)[:, :4].tolist():
            continue

        dates, closes, profits, close_rates, profit_rates, MDD, profit_product, _, path = main(selected_model_types, folder, pred_term, target_type, reinfo_th, reinfo_width)


        if profit_rates[-1] > best[0]:
            best[0] = profit_rates[-1]
            best[1] = MDD
            best[2] = profit_product
            best[3] = selected_model_types
            best[4] = reinfo_th
            best[5] = pred_term
            best[6] = reinfo_width

            best_ensemble = ensemble_models
            best_profit_rates = profit_rates

            dic = {'dates': dates, 'profits': profits, 'closes': closes, 'profit_rates': profit_rates}
            pd.DataFrame(dic).to_excel(path, index = False, encoding = 'euc-kr')

        print("중간 best >>>>")
        print(best)

        print("profit rates, MDD, profit_product, ensemble, reinfo, pred_term, width: ",
              profit_rates[-1], MDD, profit_product, selected_model_types, reinfo_th, pred_term, reinfo_width)

        results.append([ensemble_models, reinfo_th, pred_term, reinfo_width, profit_rates[-1], MDD, profit_product])
        results_df = pd.DataFrame(np.array(results), columns=['ensemble', 'self-reflection', 'pred_term', 'width', 'profit_rates', 'MDD', 'profit_product'])

        print("중간 평균 >>>>")
        print(results_df['profit_rates'].values.astype(dtype=np.float).mean())

        results_df.to_csv(result_path, index=False)

print("최종 best >>>> ")
print(best)

results_df = pd.DataFrame(np.array(results),
                          columns=['ensemble', 'self-reflection', 'pred_term', 'width', 'profit_rates', 'MDD',
                                   'profit_product'])
print("최종 평균 >>>>")
print(results_df['profit_rates'].values.astype(dtype=np.float).mean())

plt.figure(figsize=(10, 6))
plt.plot(dates, close_rates, color='r', linewidth=3.0, label='kospi200 f index')
ensemble_name = 'ensemble ' + best_ensemble
plt.plot(dates, best_profit_rates, color='b', linewidth=3.0, label=ensemble_name)
plt.legend()

ax = plt.axes()
ax.xaxis.set_major_locator(ticker.MultipleLocator(300))

plt.xticks(rotation=30)

plt.show()



nothing done! in this preprocessing

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F958571508> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001F9580B8A08>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F94A488DC8> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F958571508>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F9584CC248> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F94A488DC8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F9584CC7C8> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F9584CC248>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0

c:\users\user\anaconda3\envs\test\lib\site-packages\numpy\lib\function_base.py:390: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\users\user\anaconda3\envs\test\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F958571508> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001F9580B8A08>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F94A488DC8> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F958571508>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F9584CC248> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F94A488DC8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F9584CC7C8> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F9584CC248>).
2023-10-15 수익률: 1.0047954843750002
nothing done! in this preprocessing

Two checkpoint references resolved to different objects (<tensorflow.python.

2023-11-15 수익률: 0.9973701760416669
nothing done! in this preprocessing
2023-11-30 수익률: 1.0011580437499998
nothing done! in this preprocessing
2023-12-15 수익률: 1.0294843666666667
nothing done! in this preprocessing
2023-12-31 수익률: 1.0118121604166666
nothing done! in this preprocessing


2024-01-15 수익률: 0.9957882177083335
nothing done! in this preprocessing
2024-01-31 수익률: 0.9928214156250003
nothing done! in this preprocessing
2024-02-15 수익률: 0.9855961135416668
nothing done! in this preprocessing
2024-02-29 수익률: 0.9982949718749999
nothing done! in this preprocessing
2024-03-15 수익률: 1.001337958333333
nothing done! in this preprocessing


...

2024-10-15 수익률: 1.0119442666666667
최종 수익률 1.933732937499995
복리누적: 1.0955230295113791
MDD:  0.419431457252244
앙상블실험_random/앙상블실험_random_2023-10-16~2024-10-24_천2백투자_slippage0.0_losscut0.005/9시거래없음_ensemble_11C_reinfo_0.17_width_34_10C_30HL_5HL_.xlsx
중간 best >>>>
[1.933732937499995, 0.419431457252244, 1.0955230295113791, ['10C', '30HL', '5HL'], 0.17, 11, 34]
profit rates, MDD, profit_product, ensemble, reinfo, pred_term, width:  1.933732937499995 0.419431457252244 1.0955230295113791 ['10C', '30HL', '5HL'] 0.17 11 34
중간 평균 >>>>
1.933732937499995
nothing done! in this preprocessing

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F960439748> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001F960204A48>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F95E821988> and <tensorflow.python.keras.layers.core.Dense object at 0x0000


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F96047C4C8> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F96076D7C8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F96071DBC8> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F96047C4C8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F96071DF88> and <tensorflow.python.keras.layers.core.Dense object at 0x000001F96071DBC8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F96076D7C8> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001F960439988>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001F96047C4C8> and <tensorflow.p

2023-11-30 수익률: 1.0031173624999998
nothing done! in this preprocessing
2023-12-15 수익률: 1.0232343479166666
nothing done! in this preprocessing
2023-12-31 수익률: 1.016959805208333
nothing done! in this preprocessing
2024-01-15 수익률: 1.0102066083333334
nothing done! in this preprocessing
2024-01-31 수익률: 0.9960207333333334
nothing done! in this preprocessing
2024-02-15 수익률: 0.9962359177083333
nothing done! in this preprocessing
2024-02-29 수익률: 1.0028368583333331
nothing done! in this preprocessing
2024-03-15 수익률: 1.0078343906249998
nothing done! in this preprocessing
2024-03-31 수익률: 0.9986177145833331
nothing done! in this preprocessing


2024-04-15 수익률: 1.0050396624999998
nothing done! in this preprocessing
2024-04-30 수익률: 0.9863995833333333
nothing done! in this preprocessing
2024-05-15 수익률: 0.9863650447916672
nothing done! in this preprocessing
2024-05-31 수익률: 1.0106982375000002
nothing done! in this preprocessing
2024-06-15 수익률: 1.0077022260416664
nothing done! in this preprocessing
2024-06-30 수익률: 0.9954641093750001
nothing done! in this preprocessing
2024-07-15 수익률: 1.0156377791666669
nothing done! in this preprocessing


KeyboardInterrupt: 